#### 00. Install dependancies

In [ ]:
%%capture

# Core training libraries
!pip install -q \
    transformers==4.44.2 \
    datasets==2.20.0 \
    tokenizers==0.19.1 \
    accelerate==0.34.2 \
    peft==0.12.0 \
    trl==0.9.6 \
    bitsandbytes==0.43.1 \
    evaluate==0.4.2

# Utilities
!pip install -q \
    numpy \
    pandas \
    scikit-learn \
    rich \
    pyyaml \
    python-dotenv \
    tqdm

# Evaluation (requires pydantic v2)
!pip install -q --upgrade pydantic
!pip install -q google-genai rouge-score

print(" Installation complete!")
print(" All dependencies compatible (pydantic v2 + google-genai)")


## 1. Setting Up Environment Variables (Secrets)

In [ ]:

# Create .env file with API key
import os

# Write .env file
# with open('.env', 'w') as f:
#     # Add the secrets if needed
#     f.write('GOOGLE_API_KEY=<api_key_here>\n')
#     f.write('HF_TOKEN=<api_key_here>\n')

# print(" .env file created")

 .env file created


In [6]:
# Verify it's loaded
from dotenv import load_dotenv
load_dotenv()
# Show only key names for security
try:
    with open('.env', 'r') as f:
        print("🔑 Keys in .env file:")
        print("="*60)
        for line in f:
            line = line.strip()
            if line and not line.startswith('#'):
                key = line.split('=')[0]
                value_preview = line.split('=')[1][:10] + "..." if '=' in line else ""
                print(f"  {key} = {value_preview}")
        print("="*60)
except FileNotFoundError:
    print(" .env file not found")

🔑 Keys in .env file:
  GOOGLE_API_KEY = <api_key_h...
  HF_TOKEN = <api_key_h...
